In [1]:
import pandas as pd
import string
import numpy as np
import random

In [2]:
shakespeare = pd.read_csv("Shakespeare_data.csv")
shakespeare.dropna(inplace=True)
shakespeare.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"
5,6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils
6,7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.
7,8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil


I am going to start off just using lines from the first act and the first scene of each play to see if I can get the markov model to perform on a smaller input set

In [3]:
split = shakespeare['ActSceneLine'].str.split(".",n=3,expand=True)
shakespeare['Act']= split[0]
shakespeare['Scene']=split[1]
shakespeare['Line']=split[2]
shakespeare = shakespeare.set_index('Dataline')
firstActFirstScene = shakespeare[(shakespeare['Act'] == '1') & (shakespeare['Scene'] == '1')]
firstActFirstScene


,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine,Act,Scene,Line
Dataline,,,,,,,,
4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,",1,1,1
5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,",1,1,2
6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils,1,1,3
7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.,1,1,4
8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil,1,1,5
...,...,...,...,...,...,...,...,...
107951,A Winters Tale,11.0,1.1.41,ARCHIDAMUS,Would they else be content to die?,1,1,41
107952,A Winters Tale,12.0,1.1.42,CAMILLO,"Yes, if there were no other excuse why they sh...",1,1,42
107953,A Winters Tale,12.0,1.1.43,CAMILLO,desire to live.,1,1,43


I need to remove any punctuation from the lines. I also need to be able to split each line into words

In [4]:
def Punctuation_Be_Gone(PlayerLine):
        return PlayerLine.translate(str.maketrans('','',string.punctuation)).lower()

In [5]:
unpunctuated = []
for index, row in firstActFirstScene.iterrows():
     unpunctuated.append(Punctuation_Be_Gone(row['PlayerLine']))
unpunctuated = np.array(unpunctuated)
firstActFirstScene['Unpunctuated'] = unpunctuated
firstActFirstScene

<ipython-input-5-b4784a82a8d0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firstActFirstScene['Unpunctuated'] = unpunctuated


,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine,Act,Scene,Line,Unpunctuated
Dataline,,,,,,,,,
4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,",1,1,1,so shaken as we are so wan with care
5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,",1,1,2,find we a time for frighted peace to pant
6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils,1,1,3,and breathe shortwinded accents of new broils
7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.,1,1,4,to be commenced in strands afar remote
8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil,1,1,5,no more the thirsty entrance of this soil
...,...,...,...,...,...,...,...,...,...
107951,A Winters Tale,11.0,1.1.41,ARCHIDAMUS,Would they else be content to die?,1,1,41,would they else be content to die
107952,A Winters Tale,12.0,1.1.42,CAMILLO,"Yes, if there were no other excuse why they sh...",1,1,42,yes if there were no other excuse why they should
107953,A Winters Tale,12.0,1.1.43,CAMILLO,desire to live.,1,1,43,desire to live


In [6]:
# function for appending a new value to dictionaries
# if key already has a value then it is appended to the list of possible values 
# otherwise create the key val pair
def AppendDict(dictToAppendTo,key,val):
    if key not in dictToAppendTo:
        dictToAppendTo[key]=[val]
    else:
        dictToAppendTo[key].append(val)

In [7]:
def CreateProbDictFromFreqDict(listOfVals):
    probDict = dict()
    for word in listOfVals:
        probDict[word]=probDict.get(word,0)+1
    for key, val in probDict.items():
        probDict[key] = val/len(listOfVals)
    return probDict
    

In [8]:
firstWord = dict()
secondWord = dict()
stateTransitions = dict()
def Second_Order_Markov(df,columnName,firstWord,secondWord,stateTransitions):
   
    
    for index, row in df.iterrows():
        # split each line into words 
        # split returns a list of strings
        words = row[columnName].split()
        numWordsInLine = len(words)
        iterator = 0
        for word in words:
            # case where it is the first word of a line
            if iterator == 0:
                # updates the number of occurences of a first word
                # returns a value of zero if the key value pair dne
                firstWord[word] = firstWord.get(word, 0)+1
            else:
                previous_word = words[iterator-1]
                # case where we have reached the end of a line
                if iterator == numWordsInLine-1:
                    # add a state transition to a terminal
                    # store transition key as a tuple of words
                    AppendDict(stateTransitions, (previous_word,word), 'endl')
                    
                # case where it is the second word in a line
                # this means that there is now a previous word
                if iterator == 1:
                    # add second word to second word dictionary
                    AppendDict(secondWord,previous_word,word)
                    
                # case where there is a previous word and a previous previous word
                else:
                    # add a transition from previous previous word and previous word 
                    # to the current word using statTransitions dict
                    AppendDict(stateTransitions,(words[iterator-2],previous_word),word)
           
            iterator +=1
    # need to find probabilities for firstWord, secondWord, and for stateTransition
    for key, val in firstWord.items():
        # normalizes based off of frequency of occurences of a word
        firstWord[key]=val/sum(firstWord.values())
        
    # changes each dictionary value to a dictionary of probabilities of each transition on a given word
    for key, vals in secondWord.items():
        secondWord[key] = CreateProbDictFromFreqDict(vals)
        
    # changes each dictionary value to a dictionary of probabilities of each transition on a given word
    for key, vals in stateTransitions.items():
        stateTransitions[key] = CreateProbDictFromFreqDict(vals)
        
    #print(firstWord)
    #print(secondWord)
    #print(stateTransitions)
                
            
        
Second_Order_Markov(firstActFirstScene,"Unpunctuated",firstWord,secondWord,stateTransitions)

    

Now that the model has been trained, it is time to predict the next word given a sequence of words and generate new text.

In [9]:
def Gen_Text(numSentences,firstWord,secondWord,stateTransitions):
    for i in range(numSentences):
        line = ""
        firstword = random.choices(list(firstWord.keys()),weights=list(firstWord.values()),k=1)[0]
        secondword = random.choices(list(secondWord[firstword].keys()),weights=list(secondWord[firstword].values()),k=1)[0]
        line = line + firstword + " " + secondword
        while True:
            nextword = random.choices(list(stateTransitions[firstword,secondword].keys()),weights=list(stateTransitions[firstword,secondword].values()),k=1)[0]
            if nextword == 'endl':
                break
            line = line + " " + nextword
            firstword = secondword
            secondword = nextword
        print(line)
        

In [10]:
Gen_Text(25,firstWord,secondWord,stateTransitions)

is it done
letters yourself and so i leave you but riding forth to air yourself
good uncle let this end
little he always loved our sister most and thought to work i know she taketh most delight
winds the heavens than our courtiers
early and late debating to and fro
found it to his majesty
buried this sigh in wrinkle of a commonweal
means to thee to view
expects my coming there to see
embassies that they
loved as my cousin shallow
meanwhile i am not partial to infringe our laws
somebody had heard her talk yesterday as i am but
well sir be better employed and be you have some part of life had not their own blood did sir walter see
subject makes old hearts fresh they that master so their blood
traffics thy god confound thee
patron of virtue romes best champion
herself the glory of her sorrows takes all
receive him then to falconbridge the young baron
take it in any reason
swift as a paradise to me
suum cuique is our commission
unstanched wench
gentle son edward thou wilt be like a soldier

In [27]:
def predict_text(words,stateTransitions):
    line = ""
    wordlist = Punctuation_Be_Gone(words).split()
    if len(wordlist)==1:
        firstword = wordlist[0]
        secondword = random.choices(list(secondWord[firstword].keys()),weights=list(secondWord[firstword].values()),k=1)[0]
    else:
        firstword = wordlist[len(wordlist)-2]
        secondword = wordlist[len(wordlist)-1]
    line = words + " " + firstword + " " + secondword
    while True:
        nextword = random.choices(list(stateTransitions[firstword,secondword].keys()),weights=list(stateTransitions[firstword,secondword].values()),k=1)[0]
        if nextword == 'endl':
            break
        line = line + " " + nextword
        firstword = secondword
        secondword = nextword
    print(line)

In [33]:
predict_text("so shaken as", stateTransitions)
predict_text("if the king", stateTransitions)
predict_text("on crutches", stateTransitions)

so shaken as shaken as we thought
if the king the king
on crutches on crutches till he had deserving to command the prince your brother did employ my father left me like you the noblest that survives
